<a href="https://colab.research.google.com/github/2303a52449/GEN-AI/blob/main/GENAI_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Design a multilayer ANN architecture according to the requirements shown below.
Train, test, save (.h5) and deploy the model to predict the housing price using Keras deep
learning library
2. (1 ponto) Calculate training and testing error metrics
3. (1 ponto) Build the application by loading the saved ANN model.
Tabela 1: ANN Architecture
Layer Neurons Activation Function
Hidden Layer - 1 15 tanh
Hidden Layer - 2 20 tanh
Hidden Layer - 3 15 tanh
Tabela 2: Training Parameters
loss function epochs batch size error metric Optimizer
Mean Square Error 100 16 Mean Square Error SGD
Dataset: https://drive.google.com/file/d/1AcdENlVm5dccNyo_vgdMbneX8YVvH5R3/view?
usp=sharing


In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer

# Load dataset
data = pd.read_csv("/content/Housing.csv")

# Assuming the last column is the target variable
y = data.iloc[:, -1].values
X = data.iloc[:, :-1].values

# Convert string columns in X to numerical using Label Encoding or One-Hot Encoding if needed
for col in data.columns[:-1]:  # Iterate over all columns except the last one (target)
    if data[col].dtype == 'object':  # Check if the column is of object type (string)
        # Create a LabelEncoder object
        label_encoder_X = LabelEncoder()

        # Fit the encoder to the column and transform it
        X[:, data.columns.get_loc(col)] = label_encoder_X.fit_transform(X[:, data.columns.get_loc(col)])


# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder to the target variable and transform it
y = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical features (assuming they contain strings)
categorical_features = [i for i, col in enumerate(data.columns[:-1]) if data[col].dtype == 'object']

# Create a ColumnTransformer to handle categorical and numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), [i for i in range(X_train.shape[1]) if i not in categorical_features]),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features),
    ])

# Apply preprocessing to training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Define model
model = Sequential([
    Dense(15, activation='tanh', input_shape=(X_train.shape[1],)),
    Dense(20, activation='tanh'),
    Dense(15, activation='tanh'),
    Dense(1)  # Output layer
])

# Compile model
model.compile(optimizer=SGD(), loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)
train_mse = mean_squared_error(y_train, train_pred)
test_mse = mean_squared_error(y_test, test_pred)

print(f"Training MSE: {train_mse:.4f}")
print(f"Testing MSE: {test_mse:.4f}")

# Save model
model.save("housing_price_model.h5")

# Load and deploy model
def predict_house_price(input_data):
    loaded_model = load_model("housing_price_model.h5")
    input_data = preprocessor.transform([input_data]) # Use preprocessor to transform input data
    # ---CHANGE---: Inverse transform the prediction if the target was encoded
    prediction = loaded_model.predict(input_data)
    return label_encoder.inverse_transform(prediction.astype(int))


# Example usage
# input_example = X_test[0]  # Replace with actual input
# prediction = predict_house_price(input_example)
# print(f"Predicted Price: {prediction[0][0]}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 1.0544 - val_loss: 0.7509
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.5693 - val_loss: 0.7318
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5736 - val_loss: 0.6909
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5230 - val_loss: 0.6969
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5332 - val_loss: 0.6711
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4880 - val_loss: 0.6519
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5183 - val_loss: 0.6513
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5372 - val_loss: 0.6846
Epoch 9/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5501 - val_loss: 0.6440
Epoch 10/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5303 - val_loss: 0.6410
Epoch 11/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5135 - val_loss: 0.6697
Epoch 12/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.

Training MSE: 0.4334
Testing MSE: 0.6391
